In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import irtemp
import edge_detection as ed
import pixel_analysis as pa
import pandas as pd

## PART A:
### The temperature profile of the samples and plate is determined by detecting the edges, filling and labeling them, and monitoring the temperature at their centroids.

#### Use the function 'edge_detection.input_file' to load the input file

In [ ]:
frames = ed.input_file('../musicalrobot/data/10_17_19_PPA_Shallow_plate.tiff')
plt.imshow(frames[0])

In [ ]:
len(frames)

#### Crop the input file if required to remove the noise and increase the accuracy of edge detection

In [ ]:
crop_frame = []
for frame in frames:
    crop_frame.append(frame[30:90,60:110])
plt.imshow(crop_frame[0])
plt.colorbar()

#### Use the wrapping function edge_detection.inflection_temp 

# sample temperature by multiple pixel calculation

In [ ]:
from scipy.ndimage.morphology import binary_fill_holes
from skimage.measure import label
from skimage.measure import regionprops
from skimage.morphology import remove_small_objects
from skimage import feature
from skimage.draw import circle

In [ ]:
crop_frame[0]=crop_frame[700]

In [ ]:
for size in range(15, 9, -1):
    for thres in range(1500, 900, -100):
        edges = feature.canny(crop_frame[0]/thres)
        filled_samples = binary_fill_holes(edges)
        cl_samples = remove_small_objects(filled_samples, min_size=size)
        labeled_samples = label(cl_samples)
        props = regionprops(labeled_samples, intensity_image=crop_frame[0])

In [ ]:
props = regionprops(labeled_samples, intensity_image=crop_frame[0])
# Initializing arrays for all sample properties obtained from regprops.
row = np.zeros(len(props)).astype(int)
column = np.zeros(len(props)).astype(int)
area = np.zeros(len(props))
radius = np.zeros(len(props))
perim = np.zeros(len(props))
intensity = np.zeros(len(props), dtype=np.float64)
plate = np.zeros(len(props), dtype=np.float64)
plate_coord = np.zeros(len(props))

c = 0
for prop in props:
    row[c] = int(prop.centroid[0])
    column[c] = int(prop.centroid[1])
            # print(y[c])
    area[c] = prop.area
    perim[c] = prop.perimeter
    radius[c] = prop.equivalent_diameter/2
    rr, cc = circle(row[c], column[c], radius = radius[c]/3)
    intensity[c] = np.mean(crop_frame[0][rr,cc])
    plate[c] = crop_frame[0][row[c]][column[c]+int(radius[c])+3]
    plate_coord[c] = column[c]+radius[c]+3
    
    c = c + 1

In [ ]:
plt.imshow(labeled_samples)

# get the range of the sample by crop_frame

In [ ]:
for sample in range(len(column)):
    a=[]
    for row in range(len(labeled_samples)):
        x=labeled_samples[row]
        get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
        number=get_indexes(sample+1,x)
        a.append(number)
        while [] in a:
            a.remove([])
        
        new_data1=[]
        for i in range(len(a)):
            new_data1.append(min(a[i]))
        
        new_data2=[]
        for i in range(len(a)):
            new_data2.append(max(a[i]))
   
    
    a_2=[]
    for dot in range(len(column)):
        e=[]
        for row in range(len(labeled_samples)):
            x=labeled_samples[row].tolist()
            get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
            number=np.sum(crop_frame[0][row][get_indexes(dot+1,x)])
            e.append(number)
        final_sum=np.sum(e)
        a_2.append(final_sum)

   
    b=[]
    for row in range(len(labeled_samples)):
        o=labeled_samples[row]
        get_indexes = lambda o, os: [i for (y, i) in zip(os, range(len(os))) if o == y]
        root=get_indexes(sample+1,o)
        if root != [] :
            b.append(row)
    crop_area = (np.max(b)-np.min(b)+1)*(max(new_data2)-min(new_data1)+1)
    envir_area = crop_area - area[sample]
    
    c=[]
    for i in range(np.max(b)-np.min(b)+1):
        for j in range(max(new_data2)-min(new_data1)+1):
            crop_temp = crop_frame[0][np.min(b)+i][min(new_data1)+j]
            c.append(crop_temp)
    crop_total = sum(c)
    envir_total_temp = crop_total - a_2[sample]
    envir_average_temp = round(envir_total_temp/envir_area)
    print("sample",sample+1,"  ", "sample_temperature",round(a_2[sample]/area[sample]) ,"row:from",np.min(b),"to",np.max(b),' ',
          "column:from",min(new_data1)   ,"to",max(new_data2),'  ', "crop_area:", crop_area,
          '  ', "envir_area:",envir_area,' ', "envir_average_temp:", envir_average_temp)

#### Plotting the locations at which the temperature was recorded

In [ ]:
# Plotting the original image with the samples
# and centroid and plate location
plt.imshow(crop_frame[0])
plt.scatter(sorted_regprops[0]['Plate_coord'],sorted_regprops[0]['Row'],c='orange',s=6)
plt.scatter(sorted_regprops[0]['Column'],sorted_regprops[0]['Row'],s=6,c='red')#the origin location of the samples
plt.title('Sample centroid and plate locations at which the temperature profile is monitored')

In [ ]:
# Plotting the temperature profile of a sample against the temperature profile
# of the plate at a location next to the sample.
plt.plot(p_temp[5],s_temp[5])
plt.ylabel('Temperature of the sample($^\circ$C)')
plt.xlabel('Temperature of the well plate($^\circ$C)')
plt.title('Temperature of the sample against the temperature of the plate')

## Part B:
* The temperature profile of the samples and the plate is obtained by summing the pixel values over individual rows and columns, finding the troughs in the array of all the column and row sums.
* The temperature profile is then obtained by monitoring the temperature value at the intersection of peak values in the column and row sums. 

#### Load the input file as frames

#### Use the function irtemp.pixel_temp to get the temperature of the samples and at plate locations next to the samples in every frame of the input video.

In [ ]:
m_df = pa.pixel_temp(crop_frame,len(crop_frame),n_columns = 3, n_rows = 3)